# ThoughtLink — Feature Engineering & Separability Analysis

Visualization of extracted EEG features and class separability.

**Features extracted (66 dimensions)**:
- Band power (4 bands x 6 channels = 24)
- Hjorth parameters (3 params x 6 channels = 18)
- Time-domain statistics (4 stats x 6 channels = 24)

In [ ]:
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.insert(0, str(Path('.').resolve().parent / 'src'))

from thoughtlink.data.loader import load_all, CLASS_NAMES
from thoughtlink.data.splitter import split_by_subject
from thoughtlink.preprocessing.eeg import preprocess_all
from thoughtlink.preprocessing.windowing import windows_from_samples
from thoughtlink.features.eeg_features import (
    extract_features_from_windows, BANDS,
    compute_band_powers, compute_hjorth, compute_time_domain,
)
from thoughtlink.viz.latent_viz import (
    plot_tsne, plot_feature_importance, plot_latent_report,
    CLASS_COLORS,
)

sns.set_theme(style='whitegrid', font_scale=1.1)
%matplotlib inline

RESULTS_DIR = Path('..') / 'results'
RESULTS_DIR.mkdir(exist_ok=True)

print('Setup complete')

## 1. Data Loading & Feature Extraction

In [ ]:
# Load and preprocess
samples = load_all()
train_samples, test_samples = split_by_subject(samples, test_size=0.2)
preprocess_all(train_samples)
preprocess_all(test_samples)

# Extract windows
X_train_win, y_train, subj_train = windows_from_samples(train_samples)
X_test_win, y_test, subj_test = windows_from_samples(test_samples)

# Extract features (66 dims: band_power + hjorth + time_domain)
X_train = extract_features_from_windows(X_train_win, include_time_domain=True)
X_test = extract_features_from_windows(X_test_win, include_time_domain=True)

print(f'Train: {X_train.shape[0]} windows, {X_train.shape[1]} features')
print(f'Test:  {X_test.shape[0]} windows, {X_test.shape[1]} features')
print(f'Classes: {CLASS_NAMES}')

## 2. t-SNE Feature Space Visualization

In [ ]:
plot_tsne(
    X_train, y_train,
    class_names=CLASS_NAMES,
    title='t-SNE of EEG Feature Space (Train Set)',
    save_path=str(RESULTS_DIR / 'feature_tsne.png'),
)

## 3. Band Power Distribution per Class

Band power in the **mu (8-13 Hz)** and **beta (13-30 Hz)** bands over motor cortex channels (FCz, CPz) are key biomarkers for motor imagery BCI.

In [ ]:
# Build feature names
bands_list = list(BANDS.keys())
channels = ['AFF6', 'AFp2', 'AFp1', 'AFF5', 'FCz', 'CPz']
hjorth_params = ['activity', 'mobility', 'complexity']
time_stats = ['mean_abs', 'std', 'zcr', 'rms']

feature_names = []
for band in bands_list:
    for ch in channels:
        feature_names.append(f'{band}_{ch}')
for param in hjorth_params:
    for ch in channels:
        feature_names.append(f'{param}_{ch}')
for stat in time_stats:
    for ch in channels:
        feature_names.append(f'{stat}_{ch}')

df = pd.DataFrame(X_train, columns=feature_names[:X_train.shape[1]])
df['class'] = [CLASS_NAMES[y] for y in y_train]

# Plot mu and beta band power for motor cortex channels
motor_features = ['mu_FCz', 'mu_CPz', 'beta_FCz', 'beta_CPz']
fig, axes = plt.subplots(1, 4, figsize=(16, 4))
colors = [CLASS_COLORS[c] for c in CLASS_NAMES]

for ax, feat in zip(axes, motor_features):
    sns.boxplot(data=df, x='class', y=feat, ax=ax, palette=colors, width=0.6)
    ax.set_title(feat, fontweight='bold')
    ax.set_xlabel('')
    ax.tick_params(axis='x', rotation=45)

fig.suptitle('Band Power Distribution — Motor Cortex Channels', fontsize=14, fontweight='bold')
fig.tight_layout()
fig.savefig(str(RESULTS_DIR / 'band_power_distribution.png'), dpi=150, bbox_inches='tight')
plt.show()

## 4. Feature Importance (Inter-Class Variance)

In [ ]:
plot_feature_importance(
    X_train, y_train,
    class_names=CLASS_NAMES,
    n_top=15,
    title='Top 15 Features by Inter-Class Variance',
)

## 5. Combined Feature Space Report

In [ ]:
fig = plot_latent_report(
    X_train, y_train,
    class_names=CLASS_NAMES,
    save_path=str(RESULTS_DIR / 'feature_space_report.png'),
)
plt.show()

## 6. Mean Feature Heatmap per Class

Normalized mean feature values per class, showing which features differentiate each motor imagery task.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Compute per-class mean of standardized features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)

class_means = np.zeros((len(CLASS_NAMES), X_scaled.shape[1]))
for i, name in enumerate(CLASS_NAMES):
    mask = y_train == i
    if mask.sum() > 0:
        class_means[i] = X_scaled[mask].mean(axis=0)

# Select top 20 most variable features for readability
variance = np.var(class_means, axis=0)
top_idx = np.argsort(variance)[::-1][:20]
top_names = [feature_names[i] if i < len(feature_names) else f'feat_{i}' for i in top_idx]

fig, ax = plt.subplots(figsize=(12, 5))
sns.heatmap(
    class_means[:, top_idx],
    xticklabels=top_names,
    yticklabels=CLASS_NAMES,
    cmap='RdBu_r', center=0,
    annot=True, fmt='.2f',
    ax=ax,
)
ax.set_title('Normalized Mean Feature Values per Class (Top 20)', fontweight='bold')
fig.tight_layout()
fig.savefig(str(RESULTS_DIR / 'feature_heatmap.png'), dpi=150, bbox_inches='tight')
plt.show()

## Summary

**Key findings**:
- **66 features** extracted per 1s EEG window (band power + Hjorth + time-domain)
- Motor cortex channels (**FCz, CPz**) in **mu/beta** bands are the most discriminative
- t-SNE shows partial cluster separation between classes, with **Relax** being the most distinct
- Hjorth parameters (activity, mobility) add complementary information to spectral features
- The feature space supports both linear (SVM) and non-linear (Random Forest) classifiers